## VQE ansatz: 001

In [1]:
import os
import numpy


import pennylane as qml
from pennylane import numpy as np
from pennylane.optimize import AdamOptimizer, GradientDescentOptimizer


import sys
sys.path.append("..")
from wordsToNumbers import Corpus
from wordsToNumbers import fibonacci_vocabulary

from wordsToQubits import put_word_on_sphere

from utils import get_corpus_from_directory, working_window, get_word_from_sphere

In [2]:
np.random.seed(73)

## Corpus

In [3]:
corpus_text = "Same old dive, same old end of the work week drink Bartender knows my name,"#but I don't mind She kicks 'em up strong, serves me up right And here I go again I'm drinkin' one, I'm drinkin' two I got my heartache medication, a strong dedication To gettin' over you, turnin' me loose On that hardwood jukebox lost in neon time My heartache medication, well it suits me fine And I'm drinkin' enough to take you off my mind I got my heartache medication"
corpus= Corpus(corpus_text)
print(corpus.prop())

nr. words:15 
nr. distinct words: 13 
len.text/len.vocab:1.1538461538461537


In [4]:
parameterize_vovabulary = fibonacci_vocabulary(corpus.vocabulary)

In [5]:
print("corpus:",corpus.split_text)

corpus: ['same', 'old', 'dive', 'same', 'old', 'end', 'of', 'the', 'work', 'week', 'drink', 'bartender', 'knows', 'my', 'name']


## Training set 

In [6]:
history_lenghth = 3

In [7]:
x,y = working_window(history_lenghth, splited_text=corpus.split_text)

In [8]:
print("len training set:", len(x))

len training set: 11


In [9]:
print(y[:10])

['same', 'old', 'end', 'of', 'the', 'work', 'week', 'drink', 'bartender', 'knows']


## Circuit

In [10]:
dev = qml.device("default.qubit", wires=history_lenghth+1)

# circuit initializer
def circuit_initializer(words):
    for i in range(len(words)):
        put_word_on_sphere(words[i], qubit=i)

In [11]:
def layer_type1(param, wires=[0,1,2]):

    qml.CRZ( param[0], wires=[0,3])
    qml.CRX( param[1], wires=[0,3])
    qml.CRY( param[2], wires=[0,3])

    qml.CRZ( param[3], wires=[1,3])
    qml.CRX( param[4], wires=[1,3])
    qml.CRY( param[5], wires=[1,3])

    qml.CRZ( param[6], wires=[2,3])
    qml.CRX( param[7], wires=[2,3])
    qml.CRY( param[8], wires=[2,3])

    qml.CNOT(wires=[0,1])
    qml.RX( param[9], wires=0)
    qml.RY( param[10], wires=0)

    qml.CNOT(wires=[1,2])
    qml.RX( param[11], wires=1)
    qml.RY( param[12], wires=1)
    
    qml.CNOT(wires=[2,3])
    qml.RX( param[13], wires=2)
    qml.RY( param[14], wires=2)

    qml.CNOT(wires=[3,0])
    qml.RX( param[15], wires=3)
    qml.RY( param[16], wires=3)


In [12]:
@qml.qnode(dev)
def next_gen(params, x, obs='z'):
    """
    obs:'z', 'x' or 'y'
    """

    # initialize the circuit 
    circuit_initializer(x)

    # 
    for param in params:
        layer_type1(param, wires=[0,1,2])
    #circuit_initializer(x) # just for a test 


    # measure 
    if obs=='z':
        return  qml.expval(qml.PauliZ(3))
    if obs=='x':
        return qml.expval(qml.PauliX(3))
    if obs=='y':
        return qml.expval(qml.PauliY(3))



In [13]:
x_vec=[ parameterize_vovabulary[w] for w in x[0]]
print("x:",x[0])
print("x_vec:",x_vec)

x: ['same', 'old', 'dive']
x_vec: [[0.885600611124963, -0.33333333333333326, 0.32341992280003534], [0.8319539402538132, 0.16666666666666663, -0.5292209968608358], [-0.0, -1.0, -0.0]]


In [14]:
x_vec=[ parameterize_vovabulary[w] for w in x[0]]
print("x:",x[0])
print("x_vec:",x_vec)

params= np.random.uniform(size=(1, 17), requires_grad=True)

print("example tensor:", x_vec )
print("\n\n")
print(qml.draw(next_gen)(params,x_vec,obs='z'))

x: ['same', 'old', 'dive']
x_vec: [[0.885600611124963, -0.33333333333333326, 0.32341992280003534], [0.8319539402538132, 0.16666666666666663, -0.5292209968608358], [-0.0, -1.0, -0.0]]
example tensor: [[0.885600611124963, -0.33333333333333326, 0.32341992280003534], [0.8319539402538132, 0.16666666666666663, -0.5292209968608358], [-0.0, -1.0, -0.0]]



 0: ──RY(-5.04)──RZ(-0.36)──╭C──────────╭C──────────╭C─────────────────────────────────────────────╭C───────────RX(0.311)───RY(0.404)─────────────────────────────╭X────────────────────────┤     
 1: ──RY(-4.15)──RZ(0.198)──│───────────│───────────│───────────╭C─────────╭C──────────╭C──────────╰X──────────────────────────────────╭C───RX(0.597)──RY(0.241)──│─────────────────────────┤     
 2: ──RY(-1.57)──RZ(1.57)───│───────────│───────────│───────────│──────────│───────────│───────────╭C──────────╭C──────────╭C──────────╰X──╭C──────────RX(0.906)──│───RY(0.159)─────────────┤     
 3: ────────────────────────╰RZ(0.643)──╰RX(0.539)──╰RY(0.512)──

In [15]:
pred_vector=[ next_gen(params,x_vec, obs=o) for o in ['x', 'y', 'z']]

## Learning

In [16]:
def pred_target_distance(pred_vector, target):

    distance= np.linalg.norm(pred_vector-target)

    #np.sqrt((pred_vector[0]-target[0])*(pred_vector[0]-target[0])+
    #                 (pred_vector[1]-target[1])*(pred_vector[1]-target[1])+
    #                 (pred_vector[2]-target[2])*(pred_vector[2]-target[2]))

    return distance

In [17]:
print("target word:",y[0])
print("target vector:",parameterize_vovabulary[y[0]] )
print("prediction:",pred_vector)
pred_word= get_word_from_sphere(pred_vector, parameterize_vovabulary)
print("prediction word:",pred_word)
distance= pred_target_distance(np.array(pred_vector), np.array(parameterize_vovabulary[y[0]]))
print("distance:", distance)

target word: same
target vector: [0.885600611124963, -0.33333333333333326, 0.32341992280003534]
prediction: [tensor(0.15143019, requires_grad=True), tensor(0.02224266, requires_grad=True), tensor(-0.10865775, requires_grad=True)]
prediction word: old
distance: 0.9231097540604356


In [18]:
distance= pred_target_distance( np.array(parameterize_vovabulary[y[0]]) ,np.array(parameterize_vovabulary[y[0]]))
print(distance)

0.0


In [19]:
def cost( par , x, y):

    predictions = [[next_gen(par, w_input,obs='x') ,next_gen(par,w_input,obs='y') ,next_gen(par,w_input,obs='z') ] for w_input in x ]

    c=0.0
    for i in range(len(predictions)):
        c = c+ pred_target_distance(np.array(predictions[i]), y)

    c=c/len(predictions)

    return np.array(c)

In [20]:
def accuracy(predictions, y):
    pred_words=[ get_word_from_sphere(p_v, parameterize_vovabulary) for p_v in predictions]
    target_words=[ get_word_from_sphere(p_v, parameterize_vovabulary) for p_v in y]
    ac=0
    for i in range(len(pred_words)):
        if pred_words[i]==target_words[i]:
            ac=ac+1
    return ac/len(pred_words)

## Training:

In [21]:
## shuffling data 
index = np.array([ i for i in range(len(x))])
print(index)

[ 0  1  2  3  4  5  6  7  8  9 10]


In [22]:
X_train= []
Y_train= []
Y_train_w= []

for i in range(len(x)):
    vec = [parameterize_vovabulary[w] for w in x[i]]
    X_train.append(vec)
    Y_train.append(parameterize_vovabulary[y[i]])
    Y_train_w.append(y[i])
    
X_train= np.array(X_train)
Y_train= np.array(Y_train)

In [23]:
def iterate_batches(X,Y, batch_size):
    X1 = [torch.reshape(x[0], (1, 2 ** (len(spec.latent_qubits) + len(spec.trash_qubits)))) for x in X]
    X2 = []
    for i in range(len(X1)):
        X2.append([X1[1], X[i][1]])
    X = X2
    random.shuffle(X)

    batch_list = []
    batch = []
    for x in X:
        if len(batch) < batch_size:
            batch.append(x)

        else:
            batch_list.append(batch)
            batch = []
    if len(batch) != 0:
        batch_list.append(batch)
    return batch_list

In [24]:
## Model parameters 

num_layers= 1
layer_param= 17
#params = np.random.uniform(size=(2, layer_param), requires_grad=True)

learning_rate= 0.6
opt = AdamOptimizer(learning_rate, beta1=0.9, beta2=0.999)

nr_epochs= 2000#500

In [ ]:
ls_progres=[]
ac_progres=[]
for e in range(nr_epochs):
    params, ls = opt.step_and_cost(lambda p: cost( par=p, x=X_train, y=Y_train),params)

    print("Iter:{} | train_cost:{}".format(e, ls))
    ls_progres.append(ls)


    if e%10==0:
        predictions = [[next_gen(params, w_input,obs='x') ,next_gen(params,w_input,obs='y') ,next_gen(params,w_input,obs='z') ] for w_input in X_train ]
        ac=accuracy(predictions, y=Y_train)

        print("ac:",ac)
        ac_progres.append(ac)
  

Iter:0 | train_cost:3.5702909925263184
ac: 0.09090909090909091
Iter:1 | train_cost:3.4453281767406922
Iter:2 | train_cost:3.414921562527073
Iter:3 | train_cost:3.4426288339760593
Iter:4 | train_cost:3.358457695845408
Iter:5 | train_cost:3.3769059119671554
Iter:6 | train_cost:3.371524887100874
Iter:7 | train_cost:3.3754697298908725
Iter:8 | train_cost:3.373101366342141
Iter:9 | train_cost:3.3693039494235824
Iter:10 | train_cost:3.368035137818269
ac: 0.0
Iter:11 | train_cost:3.3675467859406623
Iter:12 | train_cost:3.3652313755161853
Iter:13 | train_cost:3.3623935807542833
Iter:14 | train_cost:3.3631357521694527
Iter:15 | train_cost:3.356066303168467
Iter:16 | train_cost:3.3497369503024106
Iter:17 | train_cost:3.348782364855822
Iter:18 | train_cost:3.350652756650166
Iter:19 | train_cost:3.3455656037335633
Iter:20 | train_cost:3.3363246525181776
ac: 0.0
Iter:21 | train_cost:3.3357912770418228
Iter:22 | train_cost:3.3356267059377487
Iter:23 | train_cost:3.3321247854130847
Iter:24 | train_co

## Results

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
fig = plt.figure()
plt.plot([x for x in range(0,len(ls_progres))],np.array(ls_progres),label="train loss")


plt.legend()
plt.title("loss_VQE_001_C0",)
plt.xlabel("epoch")
plt.ylabel("loss")

print("last loss:",ls_progres[-1])

In [ ]:
fig = plt.figure()
plt.plot([x for x in range(0,len(ac_progres)*10,10)],np.array(ac_progres),label="train accuracy")


plt.legend()
plt.title("accuracy_VQE_001_C0",)
plt.xlabel("epoch")
plt.ylabel("accyracy")

print("accuracy loss:",ac_progres[-1])

In [ ]:
for i in range(len(x)):
    
    words_vec = [parameterize_vovabulary[w] for w in x[i]]
    pred_vector=[ next_gen(params,words_vec, obs=o) for o in ['x', 'y', 'z']]
    pred_word= get_word_from_sphere(pred_vector, parameterize_vovabulary)
    print("{}={}".format(pred_word,y[i]))